<img src="https://www.inf.utfsm.cl/images/slides/Departamento-de-Informtica_HORIZONTAL.png" title="Title text" width="80%" />

<hr style="height:2px;border:none"/>

<H3 align='center'> MTI-PGE 2025 </H3>

<h1 align='center'>  Inteligencia Artificial y Aprendizaje Automático </h1>

<H3 align='center'> Trabajo Final </H3>

<H5 align='center'> Integrantes: </H3>

<H5 align='center'> Link al Video:  </H3>

<hr style="height:2px;border:none"/>





Link al desafío Kaggle https://www.kaggle.com/t/53be92513ca345a6a7454b1a91035358

## **Introducción**

Como en muchas otras áreas, la inteligencia artificial ha transformado significativamente el ámbito de la ciberseguridad, gracias a su capacidad para analizar grandes volúmenes de datos, modelar comportamientos complejos y detectar patrones asociados a actividades maliciosas. En particular, los enfoques basados en machine learning y deep learning han demostrado ser especialmente efectivos en la detección de anomalías en el tráfico de red, permitiendo identificar ataques sofisticados que pueden pasar inadvertidos para analistas humanos o sistemas tradicionales basados en reglas estáticas.

En esta tarea se proporciona el dataset UNSW-NB15, el cual contiene flujos de tráfico de red normal junto con múltiples categorías de ataques realistas y contemporáneos. Entre los escenarios de ataque presentes se incluyen, entre otros, DoS, Exploits, Reconnaissance, Fuzzers, Generic, Backdoors, Shellcode y Worms, representando una amplia variedad de técnicas utilizadas en entornos reales.

Cada flujo de red está descrito mediante un conjunto extenso de atributos que abarcan información básica de la comunicación, como direcciones IP y puertos de origen y destino, el protocolo de transporte utilizado (por ejemplo, TCP, UDP o ICMP), así como métricas temporales relacionadas con la duración del flujo. Adicionalmente, se incluyen características estadísticas y volumétricas, tales como el número de paquetes y bytes transmitidos, estados de conexión y otros indicadores de comportamiento relevantes para la detección de intrusiones.

El dataset permite abordar tanto clasificación binaria (tráfico normal vs. malicioso) como clasificación multiclase, diferenciando entre los distintos tipos de ataque. Su tarea consistirá en desarrollar un sistema basado en deep learning capaz de clasificar, de manera individual, los flujos de red presentes en el conjunto de pruebas (test), evaluando el desempeño del modelo en un escenario realista de detección de intrusiones. Para lo anterior usaremos el enfoque multiclase


## **Entregables**

* Se debe entregar el código utilizado en formato Jupyter notebook para poder **reproducir los resultados** presentados. El notebook debe estar seccionado y  ordenado para permitir identificar las celdas correspondientes a cada parte del trabajo y poder reproducir las predicciones subidas a Kaggle (si alguna pieza de la solución se carga pre-construida, deben incorporarse los archivos o los links necesarios para la ejecución completa del código).

* Se debe preparar un video de **10 a 15 minutos** donde se explique cómo se abordó el trabajo:

    - Se debe narrar brevemente la estrategia de solución adoptada enfocándose en aquello que los autores creen que la hará la solución ganadora.

    - Se descontarán 5 puntos por cada minuto excedido en el video.

    - En la cabecera del notebook pueden indicar un link al vide en youtube. Alternativamente pueden entregar un link a otro repositorio (e.g. Drive), pero por favor asegúrense de que podamos acceder al material. En caso de no poder acceder al video, la entrega se considerará no efectuada.

    - Se descontarán 5 puntos por cada día de atraso en la entrega.

* Se debe subir al menos una submission a la plataforma Kaggle como se ha hecho en clases. Su desempeño tendrá las siguientes bonificaciones o descuentos:

  - 1er Lugar = +10 pts.
  - 2do y 3er Lugar = +7 pts.
  - 4to Lugar = +5 pts.
  - 5to o más = +0 pts.

  - No hacer submission a Kaggle = -10 pts.
  - No superar el puntaje baseline = -5 pts.
    
* La solución debe incluir un mecanismo para evaluar el desempeño que tendrá la solución en datos nuevos, independientemente del puntaje que observe en Kaggle.

**IMPORTANTE: No olvide incluir al final de este notebook sus referencias al trabajo de otros que haya utilizado** El uso de sistemas como chatGPT, Claude, sCite u otros está permitido, pero debe indicarse cómo se usaron.

# Desarrollo
La estructura del trabajo está dividida en 4 bloques principales:

1.   Librerías
2.   Carga de los datos
3.   Modelo
4.   Validación

# Librerías



Se incorporan las librerías necesarias para ejecutar el proyecto

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import zipfile
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from keras.models import Sequential
from keras.layers import Dense, Dropout, Input, BatchNormalization, Activation
from google.colab import files
from collections import Counter


import tensorflow as tf



# Carga de los datos

El data set se obtendrá desde Kaggle, para ello será necesario cargar el archivo de credenciales legacy de kaggle, este servirá para interactuar con la API y realizar la descarga.

Si no se dispone del archivo de credenciales, se puede generar uno accediendo a la configuración de la cuenta en https://www.kaggle.com/settings/account y presionando el botón "Create Legacy API Key".

In [2]:
#Cargamos el archivo de credenciales
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"leonardofuentes","key":"a74237f1488996f3cd47a44d39289c6c"}'}

In [3]:
# El CLI de Kaggle requiere de que el archivo de credenciales se encuentre en el
# directorio .kaggle del usuario que lo ejecuta, los siguientes comandos ubican
# el archivo en el lugar y con los permisos apropiados para su uso

# Crea directorio
os.makedirs('/root/.kaggle/', exist_ok=True)

# Mueve el archivo de credenciales
!mv kaggle.json /root/.kaggle/

# Estabece los permisos de lectura y escritura
!chmod 600 /root/.kaggle/kaggle.json

In [4]:
# Probamos si la conexión a la API funciona revisando el leaderboard del desafío
# Este paso se puede omitir
!kaggle competitions leaderboard mti-2025-02-trabajo-final --show

  teamId  teamName                submissionDate              score    
--------  ----------------------  --------------------------  -------  
15082968  MTI_PGE_2025_JPI_MM_AZ  2026-01-12 00:54:16.866000  0.54024  
15062026  UNSW-MTI_baseline.csv   2025-12-23 19:18:46.510000  0.49631  
15059241  MTI KDM Active          2026-01-10 00:43:53.653000  0.19152  
15037259  Leonardo Fuentes        2026-01-12 00:40:59.066000  0.16376  


In [5]:
# Descargamos los archivos del desafío
!kaggle competitions download -c mti-2025-02-trabajo-final

# Dado que los archivos se encuentran en un empaquetado, es necesario extraerlos,
# los dejaremos en el directorio /content/data para futuros usos.
with zipfile.ZipFile('mti-2025-02-trabajo-final.zip', 'r') as zip_ref:
  zip_ref.extractall("/content/data")

  0% 0.00/140M [00:00<?, ?B/s]
100% 140M/140M [00:00<00:00, 1.66GB/s]


Una vez obtenidos los archivos con la data, entonces se procede a la carga de estos.

In [6]:
# Definimos los tipos de datos de las columnas que pueden parecer ambiguos
tipos_especificos = {
    'sport': str,
    'dsport': str,
    'ct_ftp_cmd': str
}

# Cargamos los datos de entrenamiento desde el archivo UNSW-MTI_train.csv
dataTrain = pd.read_csv('/content/data/UNSW-MTI_train.csv', dtype=tipos_especificos)

# Cargamos los datos de prueba desde el archivo UNSW-MTI_test_set.csv
dataTest = pd.read_csv('/content/data/UNSW-MTI_test_set.csv')

In [7]:
# Este paso se puede omitir

# Revisamos la carga realizada visualizando las primeras filas del dataset de
# entrenamiento y de test

print("\nPrimeras filas del dataset de entrenamiento: ",dataTrain.shape,"\n")
display(dataTrain.head())

print("\nPrimeras filas del dataset de prueba: ",dataTest.shape,"\n")
display(dataTest.head())


Primeras filas del dataset de entrenamiento:  (2100003, 49) 



,srcip,sport,dstip,dsport,proto,state,dur,sbytes,dbytes,sttl,...,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,attack_cat,Label
0,59.166.0.0,1390,149.171.126.6,53,udp,CON,0.001055,132,164,31,...,0,3,7,1,3,1,1,1,---,0
1,59.166.0.0,33661,149.171.126.9,1024,udp,CON,0.036133,528,304,31,...,0,2,4,2,3,1,1,2,---,0
2,59.166.0.6,1464,149.171.126.7,53,udp,CON,0.001119,146,178,31,...,0,12,8,1,2,2,1,1,---,0
3,59.166.0.5,3593,149.171.126.5,53,udp,CON,0.001209,132,164,31,...,0,6,9,1,1,1,1,1,---,0
4,59.166.0.3,49664,149.171.126.0,53,udp,CON,0.001169,146,178,31,...,0,7,9,1,1,1,1,1,---,0



Primeras filas del dataset de prueba:  (440044, 48) 



,srcip,sport,dstip,dsport,proto,state,dur,sbytes,dbytes,sttl,...,is_ftp_login,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,ID
0,59.166.0.9,7045,149.171.126.7,25,tcp,FIN,0.201886,37552,3380,31,...,NaN,,2,2,7,4,1,1,3,0
1,59.166.0.9,9685,149.171.126.2,80,tcp,FIN,5.864748,19410,1087890,31,...,NaN,,3,1,4,4,1,1,1,1
2,59.166.0.2,1421,149.171.126.4,53,udp,CON,0.001391,146,178,31,...,NaN,,3,5,2,7,1,1,4,2
3,59.166.0.2,21553,149.171.126.2,25,tcp,FIN,0.053948,37812,3380,31,...,NaN,,1,1,4,7,1,1,3,3
4,59.166.0.8,45212,149.171.126.4,53,udp,CON,0.000953,146,178,31,...,NaN,,2,5,2,1,1,1,2,4


In [8]:
# Revisamos los tipos de ataques cargados en el dataset se entrenamiento
print("\nCantidad de datos por categoría de ataque:")
print(dataTrain['attack_cat'].value_counts())


Cantidad de datos por categoría de ataque:
attack_cat
---                 1867614
Generic              153603
Exploits              33086
 Fuzzers              13805
DoS                   11446
 Reconnaissance        8698
 Fuzzers               5051
Analysis               2007
Reconnaissance         1759
Backdoor               1129
 Shellcode              917
Backdoors               534
Shellcode               223
Worms                   131
Name: count, dtype: int64


In [9]:
# Identificamos que existen datos mal etiquetados, existiendo espacios antes y
# despues, así como tambien en singular y plural, por lo que procedemos a
# estandarizarlos

dataTrain['attack_cat'] = dataTrain['attack_cat'].str.strip()
dataTrain['attack_cat'] = dataTrain['attack_cat'].replace("Backdoor", "Backdoors")

# Con esto quedamos con las 10 categorías indicadas por la descripción del dataset
print("Categorías de ataques en datos de entrenamiento normalizado: \n" , dataTrain['attack_cat'].value_counts())

Categorías de ataques en datos de entrenamiento normalizado: 
 attack_cat
---               1867614
Generic            153603
Exploits            33086
Fuzzers             18856
DoS                 11446
Reconnaissance      10457
Analysis             2007
Backdoors            1663
Shellcode            1140
Worms                 131
Name: count, dtype: int64


Acá es importante notar que la cantidad de registros normales, es muy superior a los registros de ataques, esto influirá en la metodología a abordar para el modelo.

In [10]:
# Guardamos los id del dataTest para usarlo en la submision a kaggle
test_ids = dataTest['ID'].copy()

In [11]:
# Para los datasets, corregimos las columnas con problemas

# Columnas con problemas conocidos
cols_fix = ['ct_flw_http_mthd', 'is_ftp_login', 'ct_ftp_cmd']

for df in [dataTrain, dataTest]:
    # Limpiar ct_ftp_cmd (strings vacíos o con espacios) reemplazandolos por NaN
    df['ct_ftp_cmd'] = (
        df['ct_ftp_cmd']
        .astype(str)
        .str.strip()
        .replace('', np.nan)
    )

    # Seteamos los valores NaN con el valor esperado por columna
    df['ct_flw_http_mthd'] = df['ct_flw_http_mthd'].fillna(0.0).astype(float)
    df['is_ftp_login'] = df['is_ftp_login'].fillna(0.0).astype(float)
    df['ct_ftp_cmd'] = df['ct_ftp_cmd'].fillna(0).astype(int)

In [12]:
# Preparación de la variable objetivo

# Transformamos la columna attack_cat de texto a números, el encoder usará las
# categorías aprendidas con el dataset de entrenamiento en el de test.
le_attack_cat = LabelEncoder()
dataTrain['attack_cat_encoded'] = le_attack_cat.fit_transform(dataTrain['attack_cat'])
# Revisamos las categorias de attack_cat
print("Categorias de ataque etiquetadas: \n")
for idx, clase in enumerate(le_attack_cat.classes_):
    print(f"{idx}: {clase}")

# Definimos la variable objetivo
y_train = dataTrain['attack_cat_encoded']

# Preparamos las variables de entrada, para ello eliminamos las columnas que no
# deben usarse como caracteristicas (Label, attack_cat y attack_cat_encoded) en
# el entrenamiento
X_train = dataTrain.drop(['Label', 'attack_cat', 'attack_cat_encoded'], axis=1)
# Y la colunma ID en los datos de test, de esta forma ambos conjuntos de datos
# quedan con 48 columnas
X_test = dataTest.drop(['ID'], axis=1)

Categorias de ataque etiquetadas: 

0: ---
1: Analysis
2: Backdoors
3: DoS
4: Exploits
5: Fuzzers
6: Generic
7: Reconnaissance
8: Shellcode
9: Worms


In [13]:
# Tranformamos las variables de tipo objeto (como proto y service) en categorías
# numéricas
categorical_cols = X_train.select_dtypes(include='object').columns
category_maps = {}

for col in categorical_cols:
    unique_vals = X_train[col].astype(str).unique()
    category_maps[col] = {v: i for i, v in enumerate(unique_vals)}

for col in categorical_cols:
    mapping = category_maps[col]

    X_train[col] = X_train[col].astype(str).map(mapping)
    X_test[col] = X_test[col].astype(str).map(mapping)

    # Categorías no vistas → -1
    X_train[col] = X_train[col].fillna(-1).astype(int)
    X_test[col] = X_test[col].fillna(-1).astype(int)


In [61]:
from sklearn.preprocessing import StandardScaler, PowerTransformer

# 1. Limpieza básica
dataTrain['attack_cat'] = dataTrain['attack_cat'].str.strip()
dataTrain['attack_cat'] = dataTrain['attack_cat'].replace("Backdoor", "Backdoors")

test_ids = dataTest['ID'].copy()

# 2. Arreglo de columnas específicas
for df in [dataTrain, dataTest]:
    df['ct_ftp_cmd'] = df['ct_ftp_cmd'].astype(str).str.strip().replace('', np.nan)
    df['ct_flw_http_mthd'] = df['ct_flw_http_mthd'].fillna(0.0).astype(float)
    df['is_ftp_login'] = df['is_ftp_login'].fillna(0.0).astype(float)
    df['ct_ftp_cmd'] = df['ct_ftp_cmd'].fillna(0).astype(int)

# 3. Encoding de la variable objetivo
le_attack_cat = LabelEncoder()
dataTrain['attack_cat_encoded'] = le_attack_cat.fit_transform(dataTrain['attack_cat'])
y_train = dataTrain['attack_cat_encoded']

# 4. Selección de características (Drop de columnas ruidosas)
drop_cols = ['Label', 'attack_cat', 'attack_cat_encoded', 'srcip', 'dstip', 'stcpb', 'dtcpb', 'trans_id']
X_train = dataTrain.drop(columns=[c for c in drop_cols if c in dataTrain.columns])
X_test = dataTest.drop(columns=[c for c in (drop_cols + ['ID']) if c in dataTest.columns])

# 5. Encoding de variables categóricas (proto, service, state)
categorical_cols = X_train.select_dtypes(include='object').columns
for col in categorical_cols:
    mapping = {v: i for i, v in enumerate(X_train[col].astype(str).unique())}
    X_train[col] = X_train[col].astype(str).map(mapping).fillna(-1).astype(int)
    X_test[col] = X_test[col].astype(str).map(mapping).fillna(-1).astype(int)

# --- CAMBIO CLAVE: DOBLE ESCALADO ---

# A. Escalado Estándar (Para el modelo Binario)
scaler_std = StandardScaler()
X_train_std = scaler_std.fit_transform(X_train)
X_test_std = scaler_std.transform(X_test)

# B. Escalado Power (Para el modelo Especialista)
# El método 'yeo-johnson' funciona con datos positivos y negativos.
# Esto ayuda a que los ataques minoritarios sean más "visibles" para la red.
scaler_pwr = PowerTransformer(method='yeo-johnson')
X_train_pwr = scaler_pwr.fit_transform(X_train)
X_test_pwr = scaler_pwr.transform(X_test)

# Para no romper tu código actual, asignamos X_train y X_test al estándar por defecto
X_train = X_train_std
X_test = X_test_std

print("Preprocesamiento completado.")
print("X_train_std preparado para Binario.")
print("X_train_pwr preparado para Especialista.")

Preprocesamiento completado.
X_train_std preparado para Binario.
X_train_pwr preparado para Especialista.


# Modelo

Como vimos anteriormente, existe una gran diferencia entre la cantidad de datos normales v/s los que corresponden a ataque, esto amerita que la solución se enfoque en un modelo en cascada, donde en primera instancia se identifique si corresponde a un tráfico normal o ataque, para posteriormente analizar el tipo de ataque.

## Modelo para Clasificación Binaria

In [62]:
from keras.callbacks import EarlyStopping

# Modelo Binario

# Mantenemos la lógica de etiquetas binarias
y_train_bin = (dataTrain['attack_cat'] != '---').astype(int)

# --- CAMBIO: Arquitectura más profunda y estable ---
model_bin = Sequential([
    Input(shape=(X_train.shape[1],)),

    Dense(256, activation='relu'), # Aumentamos neuronas para captar patrones complejos
    BatchNormalization(),
    Dropout(0.3),

    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),

    Dense(64, activation='relu'),
    BatchNormalization(),

    Dense(1, activation='sigmoid')
])

# --- CAMBIO: Agregamos EarlyStopping para evitar Overfitting ---
# Si la pérdida de validación no mejora en 5 épocas, el entrenamiento se detiene.
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

model_bin.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# --- CAMBIO: Agregamos validation_split para monitorear el rendimiento real ---
# Reducimos los pesos de clase ligeramente (de 5.0 a 3.0) para evitar excesivos falsos positivos
history_bin = model_bin.fit(
    X_train,
    y_train_bin,
    epochs=50,             # Aumentamos épocas pero confiamos en EarlyStopping
    batch_size=1024,       # Batch size más grande para estabilidad con BatchNormalization
    class_weight={0: 1.0, 1: 3.0},
    validation_split=0.2,  # <--- CRÍTICO para ver si el modelo generaliza
    callbacks=[early_stop],
    verbose=1
)

# --- CAMBIO: Código para ver el contador de clasificación (tu solicitud) ---
import numpy as np
clases, conteos = np.unique(y_train_bin, return_counts=True)
print("\nDistribución de clases en el entrenamiento:")
for cls, count in zip(clases, conteos):
    label = "Ataque (1)" if cls == 1 else "Normal (0)"
    print(f"{label}: {count} muestras")

Epoch 1/50
1641/1641 ━━━━━━━━━━━━━━━━━━━━ 16s 7ms/step - accuracy: 0.9733 - loss: 0.1097 - val_accuracy: 0.9819 - val_loss: 0.0300
Epoch 2/50
1641/1641 ━━━━━━━━━━━━━━━━━━━━ 15s 5ms/step - accuracy: 0.9911 - loss: 0.0232 - val_accuracy: 0.9819 - val_loss: 0.0320
Epoch 3/50
1641/1641 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.9912 - loss: 0.0219 - val_accuracy: 0.9822 - val_loss: 0.0300
Epoch 4/50
1641/1641 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.9912 - loss: 0.0215 - val_accuracy: 0.9815 - val_loss: 0.0324
Epoch 5/50
1641/1641 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.9913 - loss: 0.0211 - val_accuracy: 0.9827 - val_loss: 0.0298
Epoch 6/50
1641/1641 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.9915 - loss: 0.0206 - val_accuracy: 0.9840 - val_loss: 0.0270
Epoch 7/50
1641/1641 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.9916 - loss: 0.0203 - val_accuracy: 0.9829 - val_loss: 0.0279
Epoch 8/50
1641/1641 ━━━━━━━━━━━━━━━━━━━━ 11s 7ms/step - accuracy: 0.9918 - loss: 0.0200

In [44]:
# Contar etiquetas en el set de entrenamiento
clases, conteos = np.unique(y_train_bin, return_counts=True)
for cls, count in zip(clases, conteos):
    label = "Ataque" if cls == 1 else "Normal (---)"
    print(f"{label}: {count} muestras")

Normal (---): 1867614 muestras
Ataque: 232389 muestras


In [68]:
import tensorflow as tf
from keras.layers import Input, Dense, BatchNormalization, Dropout
from keras.models import Model
from sklearn.utils import class_weight
import numpy as np

# 1. Preparación de datos (VOLVEMOS A X_train_std / X_train normal)
solo_ataques_mask = (dataTrain['attack_cat'] != '---')
# IMPORTANTE: Usamos la versión de StandardScaler que nos dio 0.46
X_train_ataques = X_train_std[solo_ataques_mask]
y_train_ataques_text = dataTrain.loc[solo_ataques_mask, 'attack_cat']

le_ataques = LabelEncoder()
y_train_ataques_enc = le_ataques.fit_transform(y_train_ataques_text)
num_classes = len(le_ataques.classes_)

# 2. Pesos de clase suaves (Para no castigar tanto a Generic/Exploits)
counts = dataTrain.loc[solo_ataques_mask, 'attack_cat'].value_counts()
weights = 1.0 / (counts ** 0.5)
weights = weights / weights.min()
dict_pesos_final = {int(le_ataques.transform([k])[0]): float(v) for k, v in weights.items()}

# --- ARQUITECTURA SIMPLIFICADA (DIAMANTE) ---
inputs = Input(shape=(X_train_ataques.shape[1],))

x = Dense(256, activation='relu')(inputs)
x = BatchNormalization()(x)

x = Dense(512, activation='relu')(x) # Capa ancha para capturar patrones complejos
x = BatchNormalization()(x)
x = Dropout(0.4)(x)

x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)

outputs = Dense(num_classes, activation='softmax')(x)

model_especialista = Model(inputs=inputs, outputs=outputs)

# 3. Compilación
model_especialista.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# 4. Entrenamiento
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=12,
    restore_best_weights=True
)

model_especialista.fit(
    X_train_ataques,
    y_train_ataques_enc,
    epochs=100,
    batch_size=1024, # Batch más grande para suavizar gradientes
    validation_split=0.15,
    class_weight=dict_pesos_final,
    callbacks=[early_stop],
    verbose=1
)

Epoch 1/100
193/193 ━━━━━━━━━━━━━━━━━━━━ 13s 26ms/step - accuracy: 0.7645 - loss: 2.4039 - val_accuracy: 0.9252 - val_loss: 0.2705
Epoch 2/100
193/193 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8412 - loss: 1.5005 - val_accuracy: 0.9447 - val_loss: 0.1877
Epoch 3/100
193/193 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8459 - loss: 1.3845 - val_accuracy: 0.9488 - val_loss: 0.1653
Epoch 4/100
193/193 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8509 - loss: 1.3120 - val_accuracy: 0.9524 - val_loss: 0.1542
Epoch 5/100
193/193 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8536 - loss: 1.2687 - val_accuracy: 0.9484 - val_loss: 0.1562
Epoch 6/100
193/193 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8553 - loss: 1.2295 - val_accuracy: 0.9454 - val_loss: 0.1526
Epoch 7/100
193/193 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8542 - loss: 1.2208 - val_accuracy: 0.9540 - val_loss: 0.1373
Epoch 8/100
193/193 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8567 - loss: 1.1944 - val_ac

## Combinación de modelos para predicción final

In [69]:
# 1. Predicciones (Todo con X_test_std)
prob_binaria = model_bin.predict(X_test_std)
es_ataque = (prob_binaria > 0.50).astype(int).flatten()

probs_especialista = model_especialista.predict(X_test_std)
pred_especialista_indices = probs_especialista.argmax(axis=1)
confianza = probs_especialista.max(axis=1)

nombres_ataques = le_ataques.inverse_transform(pred_especialista_indices)
predicciones_finales_nombres = []

for i in range(len(es_ataque)):
    if es_ataque[i] == 0:
        predicciones_finales_nombres.append('---')
    else:
        # LÓGICA DE RESCATE: Si la confianza es baja (< 0.35),
        # asignamos a 'Generic' para asegurar el punto.
        if confianza[i] < 0.35:
            predicciones_finales_nombres.append('Generic')
        else:
            predicciones_finales_nombres.append(str(nombres_ataques[i]))

# --- GUARDAR ---
csv_filename = 'submission.csv'
submission = pd.DataFrame({'ID': test_ids, 'attack_cat': predicciones_finales_nombres})
submission.to_csv(csv_filename, index=False)
print(Counter(predicciones_finales_nombres))

13752/13752 ━━━━━━━━━━━━━━━━━━━━ 19s 1ms/step
13752/13752 ━━━━━━━━━━━━━━━━━━━━ 39s 3ms/step
Counter({'---': 345834, 'Generic': 61795, 'DoS': 10956, 'Fuzzers': 9732, 'Exploits': 7298, 'Reconnaissance': 3288, 'Shellcode': 536, 'Analysis': 368, 'Worms': 159, 'Backdoors': 78})


Evaluacion del modelo

In [ ]:
# Bajar archivo de predicciones para carga manual en colab, para subirlo
# directamente ejecutar la siguiente celda

files.download(csv_filename)

In [70]:
# Subir directamente a colab
!kaggle competitions submit -c mti-2025-02-trabajo-final -f submission.csv -m "Subido desde colab"

100% 4.86M/4.86M [00:00<00:00, 7.06MB/s]
Successfully submitted to MTI - 2025-02 - Trabajo Final